<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/arima/blob/main/3_feature_enginiering/1_el_metodo_shift_de_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/arima/blob/main/3_feature_enginiering/1_el_metodo_shift_de_pandas.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# El método `.shift()` de pandas


En **pandas**, el método **`.shift()`** es una herramienta fundamental para **desplazar (rezagar o adelantar) una serie de tiempo**, y por eso es clave cuando construyes **atributos meteorológicos con rezago** para modelos estadísticos o de *machine learning* (ARIMA/SARIMAX, regresión, Random Forest, SVM, etc.).



Dado nuestro trabajo con **series meteorológicas y epidemiológicas**, este método aparece en casi todas las etapas de *feature engineering* temporal.

---



## 1. ¿Qué hace `.shift()`?


In [60]:

import pandas as pd  


In [61]:
pd.Series.shift(periods=1, freq='W')


TypeError: NDFrame.shift() missing 1 required positional argument: 'self'

In [11]:
pd.DataFrame.shift(periods=1, freq='W')



TypeError: DataFrame.shift() missing 1 required positional argument: 'self'


* **`periods`**: número de pasos que se desplaza la serie



  * positivo → **rezago (lag)**
  * negativo → **adelanto (lead)**


* **`freq`**: desplaza el índice temporal (no los datos), se usa menos en ML



**Idea clave**:

> `.shift()` **no mezcla información del futuro**, por lo que evita *data leakage* (fuga de datos).

---



## 2. Ejemplo básico con una serie meteorológica

Supongamos una serie diaria de **temperatura media**:


In [29]:
import pandas as pd
num_periodos = 17
df = pd.DataFrame({
    "Fecha": pd.date_range("2025-12-01", periods=num_periodos, freq="D"),
    "temp_media": range(28, 28+17)
})
df

,Fecha,temp_media
0,2025-12-01,28
1,2025-12-02,29
2,2025-12-03,30
3,2025-12-04,31
4,2025-12-05,32
5,2025-12-06,33
6,2025-12-07,34
7,2025-12-08,35
8,2025-12-09,36
9,2025-12-10,37



# Rezago de 1 día


In [33]:
num_periodos_de_rezago = 2
df["temp_media_lag"+f'{num_periodos_de_rezago}'] = df["temp_media"].shift(num_periodos_de_rezago)




In [34]:
df

,Fecha,temp_media,temp_media_lag1,temp_media_lag2
0,2025-12-01,28,NaN,NaN
1,2025-12-02,29,28.0,NaN
2,2025-12-03,30,29.0,28.0
3,2025-12-04,31,30.0,29.0
4,2025-12-05,32,31.0,30.0
5,2025-12-06,33,32.0,31.0
6,2025-12-07,34,33.0,32.0
7,2025-12-08,35,34.0,33.0
8,2025-12-09,36,35.0,34.0
9,2025-12-10,37,36.0,35.0


El **NaN inicial** aparece porque no existe información previa.

---



# 3. `.shift()` en DataFrames meteorológicos

Con varias variables:


In [36]:

fechas = pd.date_range('2025-12-1', periods=5, freq = 'D')
df = pd.DataFrame({
    "temp": [28, 29, 30, 31, 32],
    "humedad": [70, 72, 75, 78, 80],
    "precipitacion": [0, 5, 0, 10, 2]
}, index=fechas)

df

,temp,humedad,precipitacion
2025-12-01,28,70,0
2025-12-02,29,72,5
2025-12-03,30,75,0
2025-12-04,31,78,10
2025-12-05,32,80,2



# Crear rezagos de un día


In [37]:


df_lag1 = df.shift(1)



In [38]:
df_lag1

,temp,humedad,precipitacion
2025-12-01,NaN,NaN,NaN
2025-12-02,28.0,70.0,0.0
2025-12-03,29.0,72.0,5.0
2025-12-04,30.0,75.0,0.0
2025-12-05,31.0,78.0,10.0



O agregarlos al mismo DataFrame:


In [39]:


df["temp_lag1"] = df["temp"].shift(1)
df["humedad_lag1"] = df["humedad"].shift(1)
df["precip_lag1"] = df["precipitacion"].shift(1)



In [40]:
df

,temp,humedad,precipitacion,temp_lag1,humedad_lag1,precip_lag1
2025-12-01,28,70,0,NaN,NaN,NaN
2025-12-02,29,72,5,28.0,70.0,0.0
2025-12-03,30,75,0,29.0,72.0,5.0
2025-12-04,31,78,10,30.0,75.0,0.0
2025-12-05,32,80,2,31.0,78.0,10.0



# 4. Creación sistemática de atributos con rezago



Muy útil en proyectos como **alertas tempranas epidemiológicas**:


In [41]:


for lag in [1, 3, 7, 14]:
    df[f"temp_lag{lag}"] = df["temp"].shift(lag)
    df[f"humedad_lag{lag}"] = df["humedad"].shift(lag)



In [42]:
df

,temp,humedad,precipitacion,temp_lag1,humedad_lag1,precip_lag1,temp_lag3,humedad_lag3,temp_lag7,humedad_lag7,temp_lag14,humedad_lag14
2025-12-01,28,70,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-02,29,72,5,28.0,70.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-03,30,75,0,29.0,72.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-04,31,78,10,30.0,75.0,0.0,28.0,70.0,NaN,NaN,NaN,NaN
2025-12-05,32,80,2,31.0,78.0,10.0,29.0,72.0,NaN,NaN,NaN,NaN



# Interpretación:

* `lag 1` → efecto inmediato
* `lag 7` → efecto semanal
  

---



# 5. Uso didáctico: interpretación causal



En meteorología–salud:

> “El número de casos hoy **no depende del clima de hoy**, sino del clima de **hace varios días**”.



Formalmente:

$$
\text{Casos}(t) = f(\text{Temp}(t-7), \text{Hum}(t-10), \text{Prec}(t-14))
$$



En código:


In [43]:


df["temp_lag7"] = df["temp"].shift(7)
df["humedad_lag10"] = df["humedad"].shift(10)
df["precip_lag14"] = df["precipitacion"].shift(14)



In [44]:
df

,temp,humedad,precipitacion,temp_lag1,humedad_lag1,precip_lag1,temp_lag3,humedad_lag3,temp_lag7,humedad_lag7,temp_lag14,humedad_lag14,humedad_lag10,precip_lag14
2025-12-01,28,70,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-02,29,72,5,28.0,70.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-03,30,75,0,29.0,72.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-04,31,78,10,30.0,75.0,0.0,28.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-05,32,80,2,31.0,78.0,10.0,29.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN



---

## 6. Diferencia entre `.shift()` y `.diff()`

| Método     | Qué hace                   | Uso típico                 |
| ---------- | -------------------------- | -------------------------- |
| `.shift()` | Desplaza la serie          | Rezagos, causalidad        |
| `.diff()`  | Resta valores consecutivos | Tendencia, estacionariedad |

Ejemplo combinado:


In [45]:


df["temp_cambio"] = df["temp"].diff()


In [46]:
df

,temp,humedad,precipitacion,temp_lag1,humedad_lag1,precip_lag1,temp_lag3,humedad_lag3,temp_lag7,humedad_lag7,temp_lag14,humedad_lag14,humedad_lag10,precip_lag14,temp_cambio
2025-12-01,28,70,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-02,29,72,5,28.0,70.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2025-12-03,30,75,0,29.0,72.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2025-12-04,31,78,10,30.0,75.0,0.0,28.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2025-12-05,32,80,2,31.0,78.0,10.0,29.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [47]:
df["temp_lag1"] = df["temp"].shift(1)



In [48]:
df

,temp,humedad,precipitacion,temp_lag1,humedad_lag1,precip_lag1,temp_lag3,humedad_lag3,temp_lag7,humedad_lag7,temp_lag14,humedad_lag14,humedad_lag10,precip_lag14,temp_cambio
2025-12-01,28,70,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-02,29,72,5,28.0,70.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2025-12-03,30,75,0,29.0,72.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2025-12-04,31,78,10,30.0,75.0,0.0,28.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2025-12-05,32,80,2,31.0,78.0,10.0,29.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0





# 7. `.shift()` y modelos de ML



### Matriz de entrenamiento


In [57]:


X = df[[
    "temp_lag1", "temp_lag7",
    "humedad_lag1", "humedad_lag7"
]]
X 

,temp_lag1,temp_lag7,humedad_lag1,humedad_lag7
2025-12-01,NaN,NaN,NaN,NaN
2025-12-02,28.0,NaN,70.0,NaN
2025-12-03,29.0,NaN,72.0,NaN
2025-12-04,30.0,NaN,75.0,NaN
2025-12-05,31.0,NaN,78.0,NaN


In [53]:
import numpy as np 
df['casos'] = np.random.randint(0, 10, size = 5)
y = df["casos"]



In [56]:
y

2025-12-01    3
2025-12-02    9
2025-12-03    8
2025-12-04    6
2025-12-05    8
Freq: D, Name: casos, dtype: int32


⚠️ **Siempre eliminar NaN antes de entrenar**:


In [58]:


Xy = pd.concat([X, y], axis=1)



In [59]:
Xy

,temp_lag1,temp_lag7,humedad_lag1,humedad_lag7,casos
2025-12-01,NaN,NaN,NaN,NaN,3
2025-12-02,28.0,NaN,70.0,NaN,9
2025-12-03,29.0,NaN,72.0,NaN,8
2025-12-04,30.0,NaN,75.0,NaN,6
2025-12-05,31.0,NaN,78.0,NaN,8
